In [1]:
# @title Run to download the required libraries and models
print("Downloading and installing libraries...")
%pip install ultralytics supervision roboflow ipywidgets pillow > /dev/null 2>&1
print("Done")
print("Downloading the model...")
!pip install gdown > /dev/null 2>&1
import gdown
file_id = '12q1xnr7_Enz_BwfP8G0DtAM4Z_i2KYhi'
destination = 'yolo11.pt'
gdown.download(f"https://drive.google.com/uc?id={file_id}", destination, quiet=True)
print("Done")
print("Setup complete!")


Done
Done
Setup complete!


In [25]:
# @title Run the model
import ipywidgets as widgets
from IPython.display import display, Image
from ultralytics import YOLO
import os
from google.colab import files
import torch
from PIL import Image as PILImage


device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = YOLO('yolo11.pt').to(device)

def generate_new_filename(base_name, ext):
    """Generate a new filename if the file already exists, appending (2), (3), etc."""
    count = 1
    new_name = f"{base_name}{ext}"
    while os.path.exists(f"/content/{new_name}"):
        count += 1
        new_name = f"{base_name} ({count}){ext}"
    return new_name

def predict(source_path, conf_threshold=0.25):
    try:
        image = PILImage.open(source_path)
        results = model.predict(image, conf=conf_threshold, save=True)
        output_path = results[0].save_dir
        image_name = os.listdir(output_path)[0]
        base_name, ext = os.path.splitext(image_name)
        final_image_name = generate_new_filename(base_name, ext)
        final_image_path = f"/content/{final_image_name}"
        os.rename(os.path.join(output_path, image_name), final_image_path)

        return final_image_path
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None


uploaded = files.upload()


file_name = next(iter(uploaded))


conf_slider = widgets.FloatSlider(value=0.25, min=0.1, max=1.0, step=0.05, description='Confidence:')
button = widgets.Button(description="Run YOLO")
output = widgets.Output()

def on_button_click(b):
    with output:
        output.clear_output()
        print("Processing...")
        output_image_path = predict(file_name, conf_slider.value)
        if output_image_path:
            print(f"Done! Result saved as {output_image_path}")

            display(Image(filename=output_image_path))
        else:
            print("Failed to process the image.")

button.on_click(on_button_click)

display(conf_slider, button, output)

Saving dog.jpeg to dog (8).jpeg


FloatSlider(value=0.25, description='Confidence:', max=1.0, min=0.1, step=0.05)

Button(description='Run YOLO', style=ButtonStyle())

Output()

In [11]:
from ultralytics import YOLO
from PIL import Image
import requests

model = YOLO('yolo11n.pt')
image = Image.open(requests.get('https://media.roboflow.com/notebooks/examples/dog.jpeg', stream=True).raw)
result = model.predict(image, conf=0.55)[0]


0: 640x384 1 person, 1 dog, 1 handbag, 14.6ms
Speed: 2.1ms preprocess, 14.6ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 384)
